In [3]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

In [4]:
dir = './최종데이터/'
df_product = pd.read_csv(dir + 'basic_data_edit.csv', usecols=['00.상품_URL','00.상품코드','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_review = pd.read_csv(dir + 'total_review.csv', usecols=['code','user','type','tone','problem','rating','feature','review','total_rating'],encoding='cp949')
print('done')

done


In [16]:
df_review

,code,user,type,tone,problem,rating,feature,review,total_rating
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0
...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5


In [53]:
#2개 이상의 리뷰를 단 유저들만 추출
user_review_count = df_review['user'].value_counts()
user_review_count = pd.DataFrame(user_review_count)
user_review_count = user_review_count.reset_index()
user_review_count.columns = ['user','count']

df_review_count = pd.merge(df_review,user_review_count,on='user',how='left')
df_review_count = df_review_count[df_review_count['count']>=2]
df_review_count

,code,user,type,tone,problem,rating,feature,review,total_rating,count
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0,14
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0,13
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0,6
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0,13
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0,10
...,...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5,5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0,6
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0,27
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5,10


In [54]:
# 고유 사용자, 고유 제품 갯수 확인
n_users = len(df_review_count['user'].unique())
n_movies = len(df_review_count['code'].unique())

n_users, n_movies # 610 명의 사용자가 9724개의 영화에 평점을 매김을 확인

(3166, 368)

In [55]:
A = df_review_count.pivot_table(index = 'code', columns = 'user',values = 'total_rating')
A = A.copy().fillna(0)
print(A)

user           02k  0321  0eun   0교   11  11월의온조  12최정  16살  17호빋베브봄  1등급  \
code                                                                        
A000000002839  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000004434  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005060  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005766  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005767  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
...            ...   ...   ...  ...  ...     ...   ...  ...      ...  ...   
A000000163655  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000163896  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000164211  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000165599  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000166043  0.0   0.0   0.0  0.0  4.0     0.0   0.0  0.0      0.0  0.0   

In [58]:
final_df = df_review_count[['user','code','total_rating']]
final_df

,user,code,total_rating
0,요다,A000000147339,5.0
1,올리브양,A000000147339,5.0
2,뚜비이,A000000147339,5.0
3,시래우,A000000147339,5.0
4,효지니,A000000147339,5.0
...,...,...,...
16500,겨울별,A000000148095,3.5
16501,moon219,A000000148095,5.0
16502,리파리해파리파리,A000000148095,3.0
16503,luv9,A000000148095,3.5


In [62]:
import surprise
reader = surprise.Reader(rating_scale = (1,5))

col_list = ['user','code','total_rating']
data = surprise.Dataset.load_from_df(final_df[col_list], reader)

In [63]:
#surprise KNN 기반 CF 진행

#학습
trainset = data.build_full_trainset()
option = {'name' : 'pearson'}
algo = surprise.KNNBasic(sim_options = option)

algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [112]:
name_list = final_df['user'].unique()
name_list = pd.Series(name_list)

index = name_list[name_list == '뮹뮹'].index[0]
index

194

In [113]:
name_list = final_df['user'].unique()
name_list = pd.Series(name_list)
name_list

0           요다
1         올리브양
2          뚜비이
3          시래우
4          효지니
         ...  
3161    dawoon
3162     고구맛탱이
3163      에주에주
3164     grape
3165       겨울별
Length: 3166, dtype: object

In [114]:
result = algo.get_neighbors(index,k=5)
print('해당 사용자와 가장 유사한 사용자들은?', result)
print(name_list[result])

해당 사용자와 가장 유사한 사용자들은? [132, 431, 1601, 1978, 1583]
132          유랭이
431           먕두
1601    hahalovu
1978        여름쿨톤
1583         지니요
dtype: object


In [127]:
code_list = []
for r1 in result:
    max_rating = data.df[data.df['user']==name_list[r1]]['total_rating'].max()
    cos_id = data.df[(data.df['total_rating']==max_rating)&(data.df['user']==name_list[r1])]['code'].values
    
    code_list.append(cos_id)
code_list

[array(['A000000147339', 'A000000137180', 'A000000137626', 'A000000137626'],
       dtype=object),
 array(['A000000146589', 'A000000147524', 'A000000152047', 'A000000152047',
        'A000000152047', 'A000000152047'], dtype=object),
 array(['A000000137078'], dtype=object),
 array(['A000000128966', 'A000000128966'], dtype=object),
 array(['A000000137078'], dtype=object)]

In [129]:
df_product

,00.상품_URL,00.상품코드,01.브랜드,02.상품명,03.가격,04.제품 주요 사양,05.모든 성분,06.총 평점,07.리뷰 개수,08_1.별점 1점,...,08_5.별점 5점,09_1.피부타입_건성,09_2.피부타입_복합성,09_3.피부타입_지성,10_1.피부고민_보습,10_2.피부고민_진정,10_3.피부고민_주름/미백,11_1.피부자극_없음,11_2.피부자극_보통,11_3.피부자극_있음
0,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000147339,아누아,[찐진정 토너] 아누아 어성초 77% 토너 350ml,30500,모든 피부용,약모밀추출물 정제수 12헥산다이올 글리세린 베타인 병풀추출물 체이스트트리추출물 우엉...,4.7,6601,0.00,...,0.80,0.18,0.63,0.19,0.17,0.83,0.01,0.76,0.22,0.02
1,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000137180,라운드랩,[증량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),30000,모든 피부용,정제수 부틸렌글라이콜 글리세린 펜틸렌글라이콜 프로판다이올 아이리쉬모스추출물 사탕수수...,4.8,5908,0.00,...,0.84,0.22,0.63,0.15,0.39,0.61,0.01,0.82,0.18,0.01
2,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000141704,브링그린,[세븐틴 승관 Pick] 브링그린 티트리시카 수딩토너 510ml,22000,모든 피부용,티트리추출물 정제수 부틸렌글라이콜 다이프로필렌글라이콜 병풀추출물 펜틸렌글라이콜 티트...,4.7,13352,0.01,...,0.80,0.13,0.59,0.28,0.14,0.86,0.01,0.61,0.36,0.03
3,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000150722,한율,한율 어린쑥 수분진정 2종세트(크림 15ml증정),49000,모든피부,토너 정제수 프로판다이올 글리세린 12헥산다이올 폴리글리세릴6카프릴레이트 폴리글리세...,4.8,600,0.00,...,0.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000104007,아벤느,아벤느 젠틀 토닝 로션 (본품200ml+100ml 추가 증정),27000,모든피부타입,아벤느온천수아쿠아 향료 하이드로제네이티드스타치하이드롤리세이트 피이지7글리세릴코코에이...,4.7,2988,0.01,...,0.77,0.24,0.63,0.13,0.30,0.70,0.01,0.78,0.20,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000147661,AHC,AHC 온리 포맨 로션 150ml,25000,모든 피부용,정제수 글리세린 디메치콘 나이아신아마이드 사이클로펜타실록산 세틸알코올 카바잎뿌리줄기...,3.3,3,0.00,...,0.00,0.00,1.00,0.00,0.67,0.33,0.00,0.67,0.33,0.00
699,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000166822,아크네스,아크네스 포맨 퍼펙트 올인원 크림 200ML,28000,모든 피부,정제수 글리세린 에틸헥실스테아레이트 12헥산다이올 세테아릴알코올 나이아신아마이드 페...,0.0,0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
700,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000147661,AHC,AHC 온리 포맨 로션 150ml,25000,모든 피부용,정제수 글리세린 디메치콘 나이아신아마이드 사이클로펜타실록산 세틸알코올 카바잎뿌리줄기...,3.3,3,0.00,...,0.00,0.00,1.00,0.00,0.67,0.33,0.00,0.67,0.33,0.00
701,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000166822,아크네스,아크네스 포맨 퍼펙트 올인원 크림 200ML,28000,모든 피부,정제수 글리세린 에틸헥실스테아레이트 12헥산다이올 세테아릴알코올 나이아신아마이드 페...,0.0,0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [132]:
for codes in code_list:
    for code in codes:
        print(df_product[df_product['00.상품코드']==code]['02.상품명'].item)

<bound method IndexOpsMixin.item of 0    [찐진정 토너] 아누아 어성초 77% 토너 350ml
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 1    [증량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정)
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 416    한율 빨간쌀 진액에멀젼 125ml
560    한율 빨간쌀 진액에멀젼 125ml
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 416    한율 빨간쌀 진액에멀젼 125ml
560    한율 빨간쌀 진액에멀젼 125ml
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 5    아비브 어성초 카밍 토너 스킨부스터
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 18    [증량] 브링그린 사철쑥 카밍 밸런스 토너 510ml
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 436    라네즈 피토알렉신 수분 진정 토너 (본품320ml+25mlx2ea+화장솜 증정)
580    라네즈 피토알렉신 수분 진정 토너 (본품320ml+25mlx2ea+화장솜 증정)
Name: 02.상품명, dtype: object>
<bound method IndexOpsMixin.item of 436    라네즈 피토알렉신 수분 진정 토너 (본품320ml+25mlx2ea+화장솜 증정)
580    라네즈 피토알렉신 수분 진정 토너 (본품320ml+25mlx2ea+화장솜 증정)
Name: 02.상품명, dtype: object>
<bound method IndexOpsMi